# YARN

Deze notebook verkent de mogelijkheden van het configureren van de Resource Manager binnen Hadoop.
Dit gaan we verkennen in het uitvoeren van de volgende stappen:
* Bestuderen van de Yarn-UI
* Beperken van de beschikbare cpu-cores en ram geheugen van de datanode
* Aanmaken van een extra resource type voor het aantal beschikbare GPU en GPU-memory (VRAM)
* Aangeven dat de single node van onze cluster beschikt over 1 GPU met 4 GB VRAM
* Aanmaken van een resource profile met 1 GPU en 2 GB VRAM

## Yarn UI

De UI geeft een reeks informatie over de beschikbare nodes in de cluster, de capabiliteiten van deze nodes en de applicatie die gescheduled, werkende, gefaald of geslaagd zijn.
De standaardlink voor deze UI te bekijken is: [http://localhost:8088](http://localhost:8088)
Deze ziet er uit als volgt:

![Yarn - homepage](images/ui_001.png)

waarbij de pagina met de informatie over de nodes eruit ziet als volgt:

![Yarn - node capabilities](images/ui_002.png)

## Beperken aantal cpu's en ram

De staardinstellingen voor het aantal cores en ram-geheugen dat beschikbaar is op een node is respectievelijk 8 cores en 8 GB Ram.
Dit is te veel voor de toepassingen die we momenteel gaan gebruiken en kan ook vaak niet gehaald worden door een laptop.
Om deze reden gaan we eerst dit beperken zodat we de laptop niet overbelasten.

Dit kan zowel in de yarn-site.xml of in de node-resources.xml file gedaan worden door het volgende toe te voegen.

    <property>
		  <description>Max available memory on each data node.</description>
		  <name>yarn.nodemanager.resource.memory-mb</name>
		  <value>2048</value>
	</property>

	<property>
		<description>Max available cores data node.</description>
		<name>yarn.nodemanager.resource.cpu-vcores</name>
		<value>4</value>
	</property>

**Welke waarden voor aantal cores en beschikbaar geheugen worden ingesteld door het bovenstaande te kiezen?**

<details>
  <summary>Klik hier voor het antwoord</summary>

  ![Yarn UI limited](images/limited.png)
</details>

Indien dit nog te veel is voor jouw PC/laptop, pas het nu aan naar een zelfgekozen waarde. Indien je de aanpassingen niet ziet verschijnen in de ui moet je waarschijnlijk nog de yarn restarten. Controleer met het commando jps of yarn correct opgestart is.

## Extra Resource Types

Standaard kan enkel het aantal cores en beschikbare ram-geheugen van een node bepaald worden.
Indien we ook de hoeveelheid VRAM dat beschikbaar is op een node willen geven aan de resource manager voor een optimale scheduling uit te voeren moeten we dit type resource nog aanmaken.
Het aanmaken van een resource type kan gedaan worden in yarn-site.xml of resource-types.xml door het volgende toe te voegen aan bijvoorbeeld resource-types.xml:

    <property>
        <name>yarn.resource-types</name>
        <value>vram</value>
    </property>
    <property>
        <name>yarn.resource-types.vram.units</name>
        <value>G</value>
    </property>

    <property>
        <name>yarn.resource-types.vram.minimum-allocation</name>
        <value>0</value>
    </property>

    <property>
        <name>yarn.resource-types.vram.maximum-allocation</name>
        <value>1024</value>
    </property>

**Let op:** Indien de file resource-types.xml nog niet bestaat kan je hem toevoegen door een andere xml te kopieren en alles te verwijderen behalve de configuration tags. Het bovenstaande stukje code moet tussen de configuration tags staan.

In bovenstaande stukje xml worden er drie zaken aangepast, namelijk:
* yarn.resource-types: Dit is een csv lijst met alle extra resources. In dit geval is dit enk "vram".
* yarn.resource-types.vram.units: Hier passen we de grootteorde aan van de waarden die we gebruiken voor het type vram (dit kan je doen per type via de naam van de variabele). De variabele G staat voor G of dus Gigabyte en andere opties kan je [hier](https://hadoop.apache.org/docs/stable/hadoop-yarn/hadoop-yarn-site/ResourceModel.html) vinden.
* yarn.resource-types.vram.minimum-allocation: De minimale hoeveelheid die kan toegekend worden aan een container voor het uitvoeren van een applicatie
* yarn.resource-types.vram.maximum-allocation: De maximale hoeveelheid die kan toegekend worden aan een enkele container.

Merk op dat je ook de Default Resource Calculator moet aanpassen. Dit kan gedaan worden in de capacity-scheduler.xml door de volgende waarde aan te passen van

    <property>
		  <name>yarn.scheduler.capacity.resource-calculator</name>
		  <value>org.apache.hadoop.yarn.util.resource.DefaultResourceCalculator</value>
	</property>

naar

    <property>
		  <name>yarn.scheduler.capacity.resource-calculator</name>
		  <value>org.apache.hadoop.yarn.util.resource.DominantResourceCalculator</value>
	</property>

Na het toevoegen van dit resource type moet dit ook zichtbaar zijn in de UI, bijvoorbeeld:

![Yarn UI with extra type](images/extraType.png)

Voeg nu als oefening een extra resource-type toe voor het aantal gpu's beschikbaar in een node. Dit wordt uitgedrukt in eenheden en heeft een minimum waarde van 0 en een maximum waarde van 4.

## Aangeven van de mogelijkheden van de datanode

In de vorige sectie hebben we twee nieuwe parameters toegevoegd om het mogelijk te maken van Yarn om rekening te houden met de grafische kaarten aanwezig in de verschillende nodes van de cluster en een minimum of maximum aantal dat toegekend kan worden.
Er ontbreekt echter nog 1 element, namelijk de hoeveelheid resources waarover onze node beschikt.
Dit kan gedaan worden in de node-resources.xml of yarn-site.xml.
Elke node moet dit voor zichzelf instellen in zijn eigen configuratiefiles.
Om het aantal cores en Ram-geheugen toe te voegen moet onderstaande xml toegevoegd worden:

    <property>
        <name>yarn.nodemanager.resource-type.vram</name>
        <value>4G</value>
    </property>

Voeg nu als oefening ook een stuk code toe om aan te geven dat je over twee gpu's bezit.

## Aanmaken resource profile

Om het aanvragen van een set van resources te vereenvoudigen en veiliger te maken tegen fouten kunnen er profielen aangemaakt worden om de aangevraagde resources te standaardiseren.
Hiervoor moeten we eerst het gebruik van profiles enablen. Dit kan door het volgende te plaatsen in de yarn-site.xml

	<property>
		<name>yarn.resourcemanager.resource-profiles.enabled</name>
		<value>true</value>
	</property>

Daarna kan bijvoorbeeld onderstaande json gebruikt worden om 4 profiles te maken die een verschillende configuratie vereisen.

    {
        "small": {
            "memory-mb" : 1024,
            "vcores" : 1
        },
        "default" : {
            "memory-mb" : 2048,
            "vcores" : 2
        },
        "large" : {
            "memory-mb": 4096,
            "vcores" : 4
        },
        "compute" : {
            "memory-mb" : 2048,
            "vcores" : 2,
            "gpu" : 1
        }
    }

Deze json moet in een file met naam resource-profiles.json komen.
Er is echter **1 groot nadeel** aan het gebruik van resource profiles verbonden.
Momenteel is het werken met resource profiles voor resources te reserveren enkel gegarandeerd voor de Command Line Interface.

## Rest API

De YARN (Yet Another Resource Negotiator) REST API biedt een set URI-resources waarmee je toegang krijgt tot informatie over het YARN-cluster, de nodes, applicaties en historische gegevens van applicaties. Deze API’s zijn ontworpen om ontwikkelaars en beheerders te helpen bij het beheren en monitoren van hun YARN-clusters via eenvoudige HTTP-aanroepen.

### Basisprincipes van de YARN REST API

#### URI-structuur

De URI’s voor de REST-gebaseerde webservices hebben de volgende syntaxis:
````
http://{http_address_of_service}/ws/{version}/{resourcepath}
````

* http_address_of_service: Het HTTP-adres van de service waarover je informatie wilt verkrijgen. Dit kan de ResourceManager, NodeManager, MapReduce Application Master of History Server zijn.
* version: De versie van de API’s. In de huidige release is de versie v1.
* resourcepath: Een pad dat een enkele resource of een verzameling resources definieert.

#### HTTP-aanvragen
Om een REST API aan te roepen, voert je applicatie een HTTP-operatie uit op de URI die is gekoppeld aan een resource. De YARN REST API ondersteunt voornamelijk de GET-methode, die informatie over de opgegeven resource ophaalt.

#### Beveiliging
De webservice REST API’s maken gebruik van dezelfde beveiligingsmechanismen als de web-UI. Als je clusterbeheerders filters hebben ingeschakeld, moet je je authenticeren via het opgegeven mechanisme.

#### Headers
De ondersteunde headers zijn onder andere:

* Accept: Geeft het gewenste antwoordformaat aan (bijvoorbeeld application/json of application/xml).
* Accept-Encoding: Ondersteunt momenteel alleen gzip-compressie (Accept-Encoding: gzip).

### Belangrijke API’s en hun gebruik

#### Clusterinformatie API

Deze API biedt algemene informatie over het YARN-cluster, zoals de cluster-ID, de totale en beschikbare geheugen- en CPU-resources, en de status van de ResourceManager.

Voorbeeld:
````
GET http://resourcemanager_host:port/ws/v1/cluster/info
````

#### Cluster metrics API

Deze API biedt gedetailleerde statistieken over het cluster, zoals het aantal ingediende, voltooide, lopende en mislukte applicaties, evenals informatie over geheugen- en CPU-gebruik.

Voorbeeld:
````
GET http://resourcemanager_host:port/ws/v1/cluster/metrics
````

#### Cluster nodes API

Deze API biedt informatie over alle nodes in het YARN-cluster, inclusief de status, het totale en beschikbare geheugen, en de CPU-resources van elke node.

Voorbeeld:
````
GET http://resourcemanager_host:port/ws/v1/cluster/nodes
````

#### Cluster applicaties API

Deze API biedt een lijst van alle applicaties in het YARN-cluster, inclusief details zoals de naam, gebruiker, status en starttijd van elke applicatie.

Voorbeeld:
````
GET http://resourcemanager_host:port/ws/v1/cluster/apps
````

#### Specifieke applicatie-informatie API

Deze API biedt gedetailleerde informatie over een specifieke applicatie in het YARN-cluster, zoals de naam, gebruiker, queue, en starttijd.

Voorbeeld:
````
GET http://resourcemanager_host:port/ws/v1/cluster/apps/{appId}
````

#### Voorbeeld van een JSON-response

Hier is een voorbeeld van een JSON-response voor een aanvraag naar de clusterinformatie API:

````json
{
  "clusterInfo": {
    "id": 1324053971963,
    "startedOn": 1324053971963,
    "state": "STARTED",
    "haState": "ACTIVE",
    "rmStateStoreName": "org.apache.hadoop.yarn.server.resourcemanager.recovery.NullRMStateStore",
    "resourceManagerVersion": "3.0.0-SNAPSHOT",
    "resourceManagerBuildVersion": "3.0.0-SNAPSHOT from unknown by user1 source checksum 11111111111111111111111111111111",
    "resourceManagerVersionBuiltOn": "2016-01-01T01:00Z",
    "hadoopVersion": "3.0.0-SNAPSHOT",
    "hadoopBuildVersion": "3.0.0-SNAPSHOT from unknown by user1 source checksum 11111111111111111111111111111111",
    "hadoopVersionBuiltOn": "2016-01-01T01:00Z",
    "haZooKeeperConnectionState": "ResourceManager HA is not enabled."
  }
}
````

#### Waar informatie te vinden over de APIs

Gedetailleerde documentatie over de YARN REST API’s kan worden gevonden op de officiële Apache Hadoop-website. Hier zijn enkele nuttige links:

[Hadoop YARN](https://hadoop.apache.org/docs/current/hadoop-yarn/hadoop-yarn-site/WebServicesIntro.html) - Introduction to the web services REST APIs1

[Apache Hadoop ResourceManager REST APIs](https://hadoop.apache.org/docs/current/hadoop-yarn/hadoop-yarn-site/ResourceManagerRest.html)

## Oefeningen

Hieronder staan een aantal oefeningen om met de requests package informatie op te vragen via de spark rest api. Let op dat er nog geen applicaties uitgevoerd zijn op de cluster dus niet alle oefeningen zullen informatie  In het volgende deel van deze leermodule gaan we applicaties uitvoeren op de cluster dus kan het handig zijn sommige oefeningen te herhalen op een later moment.

### Clusterinformatie ophalen

Schrijf een Python-script dat de algemene informatie van het YARN-cluster ophaalt en weergeeft. Dit moet onder andere de totale en beschikbare geheugen- en CPU-resources bevatten.

### Lijst van actieve applicaties
Schrijf een Python-script dat een lijst van alle actieve applicaties in het YARN-cluster ophaalt en weergeeft. Voor elke applicatie moet de naam, de gebruiker en de status worden weergegeven.

### Specifieke applicatie-informatie
Schrijf een Python-script dat de details van een specifieke applicatie in het YARN-cluster ophaalt en weergeeft. Het script moet de applicatie-ID als invoer nemen en details zoals de naam, gebruiker, queue, en starttijd weergeven.

### Lijst van nodes
Schrijf een Python-script dat een lijst van alle nodes in het YARN-cluster ophaalt en weergeeft. Voor elke node moet de status, het totale en beschikbare geheugen, en de CPU-resources worden weergegeven.

### Node details
Schrijf een Python-script dat de details van een specifieke node in het YARN-cluster ophaalt en weergeeft. Het script moet de node-ID als invoer nemen en details zoals de status, het totale en beschikbare geheugen, en de CPU-resources weergeven.

### Queue-informatie
Schrijf een Python-script dat de informatie van alle queues in het YARN-cluster ophaalt en weergeeft. Voor elke queue moet de naam, de capaciteit, de gebruikte capaciteit en de status worden weergegeven.

### Applicatiepogingen

Schrijf een Python-script dat de pogingen van een specifieke applicatie in het YARN-cluster ophaalt en weergeeft. Het script moet de applicatie-ID als invoer nemen en details zoals de poging-ID, starttijd en eindtijd weergeven.

### Containerinformatie
Schrijf een Python-script dat de informatie van alle containers van een specifieke applicatiepoging in het YARN-cluster ophaalt en weergeeft. Het script moet de applicatie-ID en poging-ID als invoer nemen en details zoals de container-ID, host en status weergeven.

### Logs van de laatste MapReduce-applicatie opvragen
Schrijf een Python-script dat de logs van de laatste uitgevoerde MapReduce-applicatie (via mrjob) in het YARN-cluster ophaalt en weergeeft. Beperk de log tot de laatste 100 regels om fouten op te sporen. Het script moet de applicatie-ID van de laatste MapReduce-applicatie automatisch detecteren.